In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch
from transformers import AutoTokenizer, ReformerForQuestionAnswering

In [34]:
tokenizer = AutoTokenizer.from_pretrained("google/reformer-crime-and-punishment")
model = ReformerForQuestionAnswering.from_pretrained("google/reformer-crime-and-punishment")

Some weights of the model checkpoint at google/reformer-crime-and-punishment were not used when initializing ReformerForQuestionAnswering: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing ReformerForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ReformerForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ReformerForQuestionAnswering were not initialized from the model checkpoint at google/reformer-crime-and-punishment and are newly initialized: ['reformer.encoder.layers.3.attention.self_attention.self_mask_value_float16', 'reformer.encoder.layers.3.attention.self_attention.mask_value_fl

In [35]:
question = "What is the capital of Germany?"
text = "Germany is a country based in Europe and the captial name is Berlin."

In [36]:
inputs = tokenizer(question, text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

In [26]:
outputs

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-0.3538, -0.1295, -0.4811, -0.1445, -0.3287, -0.0500, -0.2725,  0.0254,
          0.0020, -0.2791, -0.8788,  0.0760, -0.1713, -0.2015, -0.5512, -0.2112,
         -0.1154, -0.2549,  0.1483, -0.3111, -0.2290, -0.2806, -0.1326, -0.1399,
         -0.1924, -0.1691,  0.1851,  0.0780, -0.1407, -0.2611, -0.1789, -0.4415,
         -0.6127,  0.4565, -0.0589, -0.4990, -0.1128,  0.0796,  0.5344, -0.1095,
          0.1374,  0.2332, -0.1252,  0.0424, -0.0670, -0.1993, -0.4477,  0.3327,
         -0.1506, -0.3038,  0.8044, -0.0266, -0.1369,  0.1533, -0.4257, -0.6308]]), end_logits=tensor([[ 0.0217,  0.2129,  0.1305, -0.5226, -0.0407,  0.5836,  0.0063,  0.2467,
         -0.0516,  0.5053,  0.0290, -0.1077, -0.0156,  0.3999,  0.9891,  0.7713,
         -0.3043,  0.0972,  0.2414,  0.3405,  0.5636,  0.7653,  0.4877,  0.6189,
         -0.1989,  0.1969,  0.2315,  0.3782,  0.1472,  0.9732,  0.0204,  0.0952,
          1.0173, -0.4966,  0.2581,  0.531

In [37]:
answer_start_index = outputs.start_logits.argmax()

In [38]:
answer_start_index

tensor(0)

In [39]:
# answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

In [40]:
answer_end_index

tensor(53)

In [11]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]

target_start_index = torch.tensor([12])
target_end_index = torch.tensor([12])

In [12]:
target_start_index

tensor([12])

In [13]:
target_end_index

tensor([12])

In [14]:
outputs = model(**inputs, start_positions=target_start_index, end_positions=target_end_index)
loss = outputs.loss

In [15]:
loss

tensor(4.2324, grad_fn=<DivBackward0>)

In [16]:
def decode_qa_output(question, context, start_logits, end_logits, tokenizer):

    start_logits = torch.tensor(start_logits)
    end_logits = torch.tensor(end_logits)

    # Get the start and end indices with the highest logits
    start_index = start_logits.argmax()
    end_index = end_logits.argmax()

    # Get the tokens corresponding to the answer span
    tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(context))
    answer_tokens = tokens[start_index: end_index + 1]

    # Decode the answer tokens into text
    answer_text = tokenizer.convert_tokens_to_string(answer_tokens)

    # Remove special tokens or unwanted characters
    answer_text = answer_text.replace('[CLS]', '').replace('[SEP]', '').strip()

    return answer_text

In [41]:
start_logits = outputs["start_logits"]
end_logits = outputs["end_logits"]

decoded_answer = decode_qa_output(question, text, start_logits, end_logits, tokenizer)

<ipython-input-16-946d38247c86>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  start_logits = torch.tensor(start_logits)
<ipython-input-16-946d38247c86>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  end_logits = torch.tensor(end_logits)


In [42]:
start_logits.argmax()

tensor(0)

In [43]:
end_logits.argmax()

tensor(53)

In [44]:
print(decoded_answer)

Germany is a country based in Europe and the captial name is Berlin.
